In [ ]:
!pip install keras-tuner

     |████████████████████████████████| 71kB 3.8MB/s 
  Created wheel for keras-tuner: filename=keras_tuner-1.0.2-cp37-none-any.whl size=78938 sha256=66a07963e72e20f9450dfa43df21fe2eaec359097ded9e9e4be1ba4a61053b07
  Stored in directory: /root/.cache/pip/wheels/bb/a1/8a/7c3de0efb3707a1701b36ebbfdbc4e67aedf6d4943a1f463d6
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp37-none-any.whl size=15356 sha256=1ccbd9c1d9f741ffbe1b1bb84c47cf8439dc76fa5a0e02d165974a822678d0ed
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built keras-tuner terminaltables


In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [ ]:
fashion_mnist=keras.datasets.fashion_mnist

In [ ]:
(train_images,train_labels),(test_images,test_labels)=fashion_mnist.load_data()

4423680/4422102 [==============================] - 0s 0us/step


In [ ]:
train_images=train_images/255.0
test_images=test_images/255.0

In [ ]:
train_images[0].shape

(28, 28)

In [ ]:
train_images=train_images.reshape(len(train_images),28,28,1)
test_images=test_images.reshape(len(test_images),28,28,1)

In [ ]:
def build_model(hp):
  model = keras.Sequential([
      keras.layers.Conv2D(
          filters=hp.Int('con_1_filter', min_value=32, max_value=128, step=16),
          kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
          activation='relu',
          input_shape=(28,28,1)
      ),
      keras.layers.Conv2D(
          filters=hp.Int('con_2_filter', min_value=32, max_value=64, step=16),
          kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
          activation='relu'
      ),
      keras.layers.Flatten(),
      keras.layers.Dense(
          units=hp.Int('dense_1_units', min_value=32, max_value=123, step=16),
          activation='relu'
      ),
      keras.layers.Dense(10, activation='softmax')#output layer                      
  ])

  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  return model

In [ ]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [ ]:
tuner_search=RandomSearch(build_model,
                          objective='val_accuracy',
                          max_trials=5,directory='output',project_name="Mnist Fashion")

In [ ]:
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)

Trial 1 Complete [00h 10m 23s]
val_accuracy: 0.8516666889190674

Best val_accuracy So Far: 0.8516666889190674
Total elapsed time: 00h 10m 23s

Search: Running Trial #2

Hyperparameter    |Value             |Best Value So Far 
con_1_filter      |128               |80                
conv_1_kernel     |3                 |3                 
con_2_filter      |48                |32                
conv_2_kernel     |5                 |3                 
dense_1_units     |48                |48                
learning_rate     |0.001             |0.01              

Epoch 1/3
 179/1688 [==>...........................] - ETA: 10:30 - loss: 0.7004 - accuracy: 0.7493

In [ ]:
model=tuner_search.get_best_models(num_models=1)[0]

In [ ]:
model.summary()